Opponent shot selection
http://stats.nba.com/league/player/#!/oppshooting/

In [10]:
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats

import requests
import time
from scrapy.selector import Selector
from scrapy.http import HtmlResponse
from selenium import webdriver
from bs4 import BeautifulSoup
import json

chromedriver = "/Users/amos/Desktop/chromedriver"

In [11]:
# opens selenium webdriver
browser = webdriver.Chrome(executable_path= chromedriver)

In [12]:
browser.get('http://stats.nba.com/league/player/#!/oppshooting/')
#json html format
HTML_logs = browser.page_source
years = Selector(text=HTML_logs).xpath("//select[@name='Season']/option/text()").extract()

In [13]:
stats_url = 'http://stats.nba.com/stats/leaguedashplayershotlocations?College=&Conference=\
&Country=&DateFrom=&DateTo=&DistanceRange=5ft+Range&Division=&DraftPick=&DraftYear=&GameScope=&\
GameSegment=&Height=&LastNGames=0&LeagueID=00&Location=&MeasureType=Opponent&Month=0&\
OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=PerGame&Period=0&PlayerExperience=&\
PlayerPosition=&PlusMinus=N&Rank=N&Season=%s&SeasonSegment=&SeasonType=Regular+Season&\
ShotClockRange=&StarterBench=&TeamID=0&VsConference=&VsDivision=&Weight='

In [14]:
# Find Header or column names
link = stats_url % '2001-02'
browser.get(link)
HTML = browser.page_source
soup = BeautifulSoup(HTML, 'lxml')
text = soup.findAll('pre')[0].getText().encode('utf-8')
json_text = json.loads(text)
shot_ranges = json_text['resultSets']['headers'][0]['columnNames']
new_ranges = [rang.replace('-','_').replace('.','').replace(' ', '').
                  replace('+','above').lower() for rang in shot_ranges]
columns = json_text['resultSets']['headers'][1]['columnNames']
fg_list = ['FGM', 'FGA', 'FG_PCT']
headers = columns[:5]
for rang in new_ranges:
    for f in fg_list:
        headers.append('opp_' + rang + '_'+ f)
headers

[u'PLAYER_ID',
 u'PLAYER_NAME',
 u'TEAM_ID',
 u'TEAM_ABBREVIATION',
 u'AGE',
 u'opp_lessthan5ft_FGM',
 u'opp_lessthan5ft_FGA',
 u'opp_lessthan5ft_FG_PCT',
 u'opp_5_9ft_FGM',
 u'opp_5_9ft_FGA',
 u'opp_5_9ft_FG_PCT',
 u'opp_10_14ft_FGM',
 u'opp_10_14ft_FGA',
 u'opp_10_14ft_FG_PCT',
 u'opp_15_19ft_FGM',
 u'opp_15_19ft_FGA',
 u'opp_15_19ft_FG_PCT',
 u'opp_20_24ft_FGM',
 u'opp_20_24ft_FGA',
 u'opp_20_24ft_FG_PCT',
 u'opp_25_29ft_FGM',
 u'opp_25_29ft_FGA',
 u'opp_25_29ft_FG_PCT',
 u'opp_30_34ft_FGM',
 u'opp_30_34ft_FGA',
 u'opp_30_34ft_FG_PCT',
 u'opp_35_39ft_FGM',
 u'opp_35_39ft_FGA',
 u'opp_35_39ft_FG_PCT',
 u'opp_40aboveft_FGM',
 u'opp_40aboveft_FGA',
 u'opp_40aboveft_FG_PCT']

In [15]:
# Create Data frame of each regular season opponents player stats from 1996-2016 and save to csv file.

for year in years:
    print year
    link = stats_url % year
    #selenium browser opens link
    browser.get(link)
    HTML = browser.page_source
    soup = BeautifulSoup(HTML, "lxml")
    get_text = soup.findAll('pre')[0].getText().encode('utf-8')
    json_text = json.loads(get_text)

    
    df = pd.DataFrame(json_text['resultSets']['rowSet'], columns=headers) 
    save_csv = '/Users/amos/Desktop/DSI-SF-2-amoskim25/copy_capstone/datasets/opponent_shot_range/opp_shot_range%s.csv' % year
    df.to_csv(save_csv)
#     time.sleep(5*np.random.rand())


2015-16
2014-15
2013-14
2012-13
2011-12
2010-11
2009-10
2008-09
2007-08
2006-07
2005-06
2004-05
2003-04
2002-03
2001-02
2000-01
1999-00
1998-99
1997-98
1996-97


In [16]:
pd.read_csv('../datasets/opponent_shot_range/opp_shot_range1997-98.csv')

,Unnamed: 0,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,AGE,opp_lessthan5ft_FGM,opp_lessthan5ft_FGA,opp_lessthan5ft_FG_PCT,opp_5_9ft_FGM,...,opp_25_29ft_FG_PCT,opp_30_34ft_FGM,opp_30_34ft_FGA,opp_30_34ft_FG_PCT,opp_35_39ft_FGM,opp_35_39ft_FGA,opp_35_39ft_FG_PCT,opp_40aboveft_FGM,opp_40aboveft_FGA,opp_40aboveft_FG_PCT
0,0,920,A.C. Green,1610612742,DAL,34.0,12.2,20.4,0.600,2.3,...,0.321,0.0,0.8,0.000,0.0,0.8,0.000,0.1,0.6,0.100
1,1,243,Aaron McKie,1610612755,PHI,25.0,6.1,10.8,0.562,1.7,...,0.313,0.2,0.7,0.333,0.0,1.0,0.000,0.0,0.7,0.000
2,2,1425,Aaron Williams,1610612760,SEA,26.0,4.5,7.5,0.599,0.7,...,0.326,0.0,0.7,0.000,0.0,0.5,0.000,0.0,1.0,0.000
3,3,228,Adam Keefe,1610612762,UTA,28.0,8.0,13.6,0.583,1.6,...,0.395,0.0,0.8,0.000,0.0,0.3,0.000,0.0,0.7,0.000
4,4,1502,Adonal Foyle,1610612744,GSW,23.0,4.1,7.1,0.567,0.9,...,0.362,NaN,NaN,NaN,0.0,0.3,0.000,0.0,0.3,0.000
5,5,154,Adrian Caldwell,1610612742,DAL,31.0,1.0,1.0,1.000,NaN,...,0.000,NaN,NaN,NaN,0.0,1.0,0.000,NaN,NaN,NaN
6,6,673,Alan Henderson,1610612737,ATL,25.0,10.2,18.2,0.562,1.8,...,0.384,0.0,0.4,0.000,0.1,0.4,0.333,0.0,0.6,0.000
7,7,275,Allan Houston,1610612752,NYK,27.0,9.9,16.8,0.585,2.2,...,0.277,0.0,0.7,0.000,0.0,0.3,0.000,0.1,0.4,0.143
8,8,947,Allen Iverson,1610612755,PHI,23.0,13.3,22.7,0.586,2.6,...,0.325,0.1,0.9,0.125,0.0,0.4,0.000,0.0,0.6,0.000
9,9,297,Alonzo Mourning,1610612748,MIA,28.0,8.4,16.8,0.498,2.8,...,0.349,0.0,0.0,0.000,0.0,0.4,0.000,0.0,0.5,0.000
